In [ ]:
import torch
sampled_images = torch.rand(252,6,256,256,3)
subject = torch.cat([i*torch.ones(50) for i in range(4)] + [4*torch.ones(52)])
print(subject)

In [ ]:
naive_label = torch.cat([torch.cat(5*[torch.arange(150,200)]),torch.tensor([198]),torch.tensor([199])])
print(naive_label)

In [ ]:
import numpy as np
def concat_images(sampled_images, subject, naive_label):
    # all_sampled_images = torch.stack(sampled_images)  # (250+,num_samples+1,3,256,256)
    # all_subject_idx = torch.stack(subject)  # (250+,)
    # all_naive_label = torch.stack(naive_label)  # (250+,)
    all_sampled_images = sampled_images
    all_subject_idx = subject
    all_naive_label = naive_label
    # 按照subject_idx取samples
    num_subjects = 5
    
    all_sorted_unique_images = []

    for i in range(num_subjects):
        selected_images = all_sampled_images[all_subject_idx == i]
        selected_naive_labels = all_naive_label[all_subject_idx == i]

        sorted_naive_labels, indices= torch.sort(selected_naive_labels)
        sorted_images = selected_images[indices]
        # sorted_images = torch.gather(selected_images,0,indices)

        unique_sorted_naive_labels, unique_indices = np.unique(sorted_naive_labels, return_index=True)
        xx = sorted_naive_labels[unique_indices] # 应该与unique_sorted_naive_label一致
        unique_sorted_images = sorted_images[unique_indices]

        # unique_naive_labels, inverse_indices = torch.unique(selected_naive_labels, return_inverse=True)
        # unique_images = selected_images[inverse_indices]

        # sorted_unique_naive_labels, indices= torch.sort(unique_naive_labels)
        # sorted_unique_images = unique_images[indices]


        all_sorted_unique_images.append(unique_sorted_images)
    results = torch.stack(all_sorted_unique_images)
    return results


aa = concat_images(sampled_images, subject, naive_label)

In [ ]:
import torch
torch.cuda.empty_cache()

In [ ]:
normalized_shape = 1024
(normalized_shape,)

In [ ]:
normalized_shape = (normalized_shape,)
a = tuple(normalized_shape)

In [ ]:
print('\U0001F383'*40)

In [ ]:
import torch
import torch.nn as nn

c = torch.rand(5,291,1024)
norm = nn.LayerNorm(1024)
out = norm(c)

In [ ]:
print(norm.weight.shape)

In [ ]:
import torch
a = torch.randn(1, 2, 3, 4)
a.size()
# torch.Size([1, 2, 3, 4])
b = a.transpose(1, 2)  # Swaps 2nd and 3rd dimension
b.size()
# torch.Size([1, 3, 2, 4])
c = a.view(1, 3, 2, 4)  # Does not change tensor layout in memory
c.size()
# torch.Size([1, 3, 2, 4])
torch.equal(b, c)
# Flase


In [1]:
import torch
import torch.nn as nn
from pdnorm_model import PDNorm
from torch import Tensor
from dc_ldm.modules.diffusionmodules.util import conv_nd



class PDGroupNorm(PDNorm, nn.GroupNorm):
    
    def __init__(self, num_groups: int, num_channels: int, prompt_dim: int=16, eps: float = 0.00001, 
                 affine: bool = False,  # False --> super.__init__中取消初始化self.weight和self.bias
                 device=None, dtype=None) -> None:
        nn.GroupNorm.__init__(self, num_groups, num_channels, eps, affine, device, dtype)
        
        self.mlp_scale = nn.Sequential(nn.Linear(prompt_dim, num_channels), nn.SiLU())
        self.mlp_shift = nn.Sequential(nn.Linear(prompt_dim, num_channels), nn.SiLU())
    
    def forward(self, x: Tensor, prompt:Tensor) -> Tensor:
        # 在C维，分组计算mean和var  
        B,C,H,W = x.shape
        x_i_standard = torch.empty(x.shape)
        for i in range(self.num_groups):
            x_i = x[:,int(i*(C/self.num_groups)):int((i+1)*(C/self.num_groups)),:,:] # (B,c/g,h,w)
            mean = torch.mean(x_i, dim=1, keepdim=True)  
            var = torch.var(x_i, dim=1, keepdim=True, unbiased=False)  # (b,1,h,w)
            x_i_standard[:,int(i*(C/self.num_groups)):int((i+1)*(C/self.num_groups)),:,:] = \
                        (x_i-mean)/torch.sqrt(var+self.eps)  # (B,c/g,h,w)
        #####################
        scale = self.mlp_scale(prompt).reshape((B,C,1,1))  
        shift = self.mlp_shift(prompt).reshape((B,C,1,1))  # (B,prompt_dim)-->(B,C)-->(B,C,1,1)
        out = x_i_standard * scale + shift
        # out = F.group_norm(
        #     x, self.num_groups, scale, shift, self.eps)        
        return out

in_layers = nn.Sequential(
            PDGroupNorm(32,192,16),
            nn.SiLU(),
            conv_nd(2, 192, 192, 3, padding=1),
        )

In [2]:
# layer = PDGroupNorm(32, 192)
# x:(5,192,64,64) p:(5,16) emb(5,768)
x = torch.ones(5,192,64,64)
p = torch.ones(5, 16)
out0 = in_layers[0](x, p)
print(out0.shape)
out1 = in_layers[1](out0)
print(out1.shape)

In [1]:
import torch
torch.cuda.empty_cache()